In [21]:
import pandas as pd
import numpy as np
data_path = r'../Mnist/'
train_data = pd.read_csv(data_path+'mnist_train.csv', header = None)
test_data = pd.read_csv(data_path+'mnist_test.csv ', header = None)
xtrain = train_data.loc[:, 1:]
ytrain = train_data.loc[:, 0]
xtest = test_data.loc[:, 1:]
ytest = test_data.loc[:, 0]

### 每个特征值二值化，相当于特征只有0和1两种，也就是决策树的每个节点只有两个分叉
### 而一共要分出10类

In [22]:
xtrain[xtrain<128] = 0
xtrain[xtrain>=128] = 1
xtest[xtest<128] = 0
xtest[xtest>=128] = 1

e:\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
e:\python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
e:\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

In [11]:
from math import log, exp

In [30]:
def majorClass(labels):
    '''
    计算所有类中占比最大的类
    '''
    dic = {}
    for l in labels:
        if l in dic:
            dic[l] += 1
        else:
            dic[l] = 1
#  dic:
#  key: label, val: cnt
    res_dic = sorted(dic.items(), key = lambda it: it[1], reversed = True)
    return res_dic[0][0]

In [33]:
def entropy(prob_list):
    return sum([-prob*log(prob) for prob in prob_list])
def cal_H(ytrain):
    '''
    计算当前特征的熵增
    '''
    tol, dic = 0, {}
    for y in ytrain:
        if y in dic:
            dic[y] += 1
        else:
            dic[y] = 1
        tol += 1
    prob_ls = []
    for k in dic.keys():
        prob_ls.append(dic[k]/tol)
    return entropy(prob_ls)

In [36]:
def cal_D_A(xfeature, ytrain):
    featureSet = set(xfeature)
    D_A = 0
    for fea in featureSet:
        D_A += xfeature[fea == xfeature].size/xfeature.size*cal_H(ytrain[xfeature==fea])
    return D_A

In [38]:
def sel_fea(xfeas, ytrain):
    xfeas = xfeas.to_numpy()
    ytrain = ytrain.to_numpy()
    n, m = xfeas.shape
    HD = cal_H(ytrain)
    max_delt, max_fea = 0, 0
    for i in range(m):
        fea = xfeas[:, m]
        D_A = cal_D_A(fea, ytrain)
        if HD-D_A>delt:
            delt = HD-D_A
            max_fea = fea
    return max_fea, max_delt

In [66]:
nn = np.array([[1,2,3], [4,8,9]])


In [70]:
def getSubData(xtrain, ytrain, fea, a):
    '''
    获取分割后的特征和标签
    :param xtrain: 训练数据集
    :param ytrain: 标签
    :param fea: 哪维特征
    :param a: 标签值是多少
    '''
    ret_labels, ret_feas = [], [] 
    for i in range(len(xtrain)):
        if xtrain[i][fea] == a:
            ret_feas.append(xtrain[i][:fea]+xtrain[i][fea+1:])
            ret_labels.append(ytrain[i])
    return ret_feas, ret_labels

In [71]:
def createTree(xtrain,ytrain):
    '''
    建树
    :param xtrain: 训练数据
    :param ytrain: 标签
    '''
    epsilon = 0.2
    print("Begin creating tree: ")             
    clas = set([i for i in ytrain])
    if clas == 1:
        return ytrain[0]
    if len(xtrain==0):
        return majorClass(ytrain)
    max_fea, max_delt = sel_fea(xtrain, ytrain)
    if max_delt<0.2:
        return majorClass(ytrain)
    types = set([i for i in xtrain[:, max_fea]])
    tree_dic = {max_fea: {}}
    for t in types:
        rest_xtrain, rest_ytrain= getSubData(xtrain, ytrain, max_fea, t)
        tree_dic[max_fea][t] = createTree(rest_xtrain, rest_ytrain)
    return tree_dic

In [74]:
d = {2:{3:5,2:{3,8,9}}}

In [90]:
(k, v), = d.items()

In [92]:
k

2

In [91]:
v

{3: 5, 2: {3, 8, 9}}

In [ ]:
def predict(xtest, tree):
    while 